In [1]:
using PorousMaterials # Pkg.clone("https://github.com/SimonEnsemble/PorousMaterials.jl", "v0.1.1")
using CSV
using DataFrames
using TSne
using PyCall
@pyimport adjustText
import PyPlot; const plt = PyPlot # now do plt.plt[:hist] for example
using DelimitedFiles
using LinearAlgebra
using Printf
using Random
using Optim
using ProgressMeter
using JLD2
using CoherentPointDrift # https://github.com/SimonEnsemble/CoherentPointDrift.jl

In [2]:
cages = readdlm("all_cages/all_cages.txt")
cages[1]

"A11"

In [3]:
aligned_and_centered_cage(cage::AbstractString) = read_xyz("all_cages/" * cage * "_final_alignment.xyz")

aligned_and_centered_cage (generic function with 1 method)

In [4]:
# Function taken from PorousMaterials.jl
function _flood_fill!(grid::Grid, segmented_grid::Grid, 
            queue_of_grid_pts::Array{Tuple{Int, Int, Int}, 1}, 
            i::Int, j::Int, k::Int, energy_tol::Float64)
    # look left, right, up, down "_s" for shift
    for i_s = -1:1, j_s = -1:1, k_s = -1:1
        # well already know segmented_grid[i, j, k]
        if (i_s, j_s, k_s) == (0, 0, 0)
            continue
        end
    
        # index of a neighboring point
        ind = (i + i_s, j + j_s, k + k_s)
    
        # avoid out of bounds
        if any(ind .<= (0, 0, 0)) || any(ind .> grid.n_pts)
            continue
        end
        # if neighbor already assigned or un-occupiable, continue
        if segmented_grid.data[ind...] != 0
            continue
        end

        # if accessible, assign same segment ID and keep looking at surrounding points
        if grid.data[ind...] < energy_tol
            segmented_grid.data[ind...] = segmented_grid.data[i, j, k]
            # add grid pt. to queue to look at ITS surrounding neighbors
            # originally had recursive algo but get stack overflow
            push!(queue_of_grid_pts, ind)
        else
            # if not accessible, assign -1 and don't push any points to the queue
            #  (this is how the algo eventually dies out)
            segmented_grid.data[ind...] = -1
        end
    end 
    return nothing
end

_flood_fill! (generic function with 1 method)

In [5]:
# Function taken from PorousMaterials.jl
function _segment_grid(grid::Grid, energy_tol::Float64, verbose::Bool)
    # grid of Int's corresponding to each original grid point.
    # let "0" be "unsegmented"
    # let "-1" be "not occupiable" and, eventually, "not accessible"
    segmented_grid = Grid(grid.box, grid.n_pts, zeros(Int, grid.n_pts...),
        :Segment_No, grid.origin)
    segment_no = 0 
    for i = 1:grid.n_pts[1], j = 1:grid.n_pts[2], k = 1:grid.n_pts[3]
        if segmented_grid.data[i, j, k] == 0 # not yet assigned segment 
            if grid.data[i, j, k] > energy_tol # not accessible
                segmented_grid.data[i, j, k] = -1
            else # accessible
                # start a new segment!
                segment_no += 1
                # initiate a queue of grid points
                queue_of_grid_pts = [(i, j, k)] 
                while length(queue_of_grid_pts) != 0
                    # take first index in line
                    id = queue_of_grid_pts[1]
                    # assign segment number
                    segmented_grid.data[id...] = segment_no
                    # look at surroudning points, add to queue if they are also accessible
                    _flood_fill!(grid, segmented_grid, queue_of_grid_pts,
                        id..., energy_tol)
                    # handled first one in queue, remove from queue
                    deleteat!(queue_of_grid_pts, 1)
                end
            end
        else # already segmented
            nothing
        end
    end
    @assert sum(segmented_grid.data == 0) == 0 "some points were unsegmented!"
    verbose ? @printf("Found %d segments\n", segment_no) : nothing
    return segmented_grid
end

_segment_grid (generic function with 1 method)

In [65]:
xe = Molecule("Xe")
kr = Molecule("Kr")

cutoffradius = 14.0 # vdw cutoff radius
ljff = LJForceField("UFF.csv", cutoffradius=cutoffradius, mixing_rules="geometric");

In [7]:
snapshot_dim = 0.0 # find max dimension we need for snapshot to include all atoms of all cages
biggest_cage = ""

for cage in cages
    # read in aligned and centered cage
    atoms, x = aligned_and_centered_cage(cage)
    # observe max dimension so we know size of snapshot to take.
    x_span = 2.0 * maximum(abs.(x))
    if x_span > snapshot_dim # 2 b/c centered at zero
        snapshot_dim = x_span
        biggest_cage = cage
    end
    
    #
    # write to .cif for molecular simulation
    #
    # make sure with PBCs no interactions are included by padding
    box = Box([x_span + 2 * cutoffradius for blah = 1:3]..., 
              [π/2 for blah = 1:3]...)
    # when writing .cif, shift coords so they are in [0, 1]^3
    #    so in .cif center of mass is center of box.
    x_shift = [x_span + 2 * cutoffradius for blah = 1:3] / 2.0
    framework = Framework(cage, box, 
                          Atoms(atoms, x), 
                          Charges(Array{Float64, 1}(undef, 0), Array{Float64, 2}(undef, 0, 0))
                          )
    write_cif(framework, 
        @sprintf("data/crystals/%s_aligned.cif", cage))
end

println("aligned and centered cages written to .xyz in all_cages/*_aligned.xyz")
println("see data/crystals for .cif's padded by cutoffradius for molecular simulations'")
println("Largest cage: :", biggest_cage)

aligned and centered cages written to .xyz in all_cages/*_aligned.xyz
see data/crystals for .cif's padded by cutoffradius for molecular simulations'
Largest cage: :MC6


In [8]:
snapshot_dim = ceil(snapshot_dim)
println("Dimension of snapshot to fit all porous cages (Å): ", snapshot_dim)

const center_of_box = [snapshot_dim / 2.0 for d = 1:3]

const snapshot_box = Box(snapshot_dim, snapshot_dim, snapshot_dim, π/2, π/2, π/2)

Dimension of snapshot to fit all porous cages (Å): 40.0


Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 40.000000 Å. b = 40.000000 Å, c = 40.000000 Å
	Volume of unit cell: 64000.000000 Å³


In [83]:
# 100 grid points in each dimension is time consuming. Consider lowering this if you don't want to wait
const nb_grid_pts = 101
const x_grid_pts = collect(range(0, stop=snapshot_dim, length=nb_grid_pts)) .- snapshot_dim / 2.0

101-element Array{Float64,1}:
 -20.0              
 -19.6              
 -19.2              
 -18.8              
 -18.4              
 -18.0              
 -17.6              
 -17.2              
 -16.8              
 -16.4              
 -16.0              
 -15.6              
 -15.2              
   ⋮                
  15.600000000000001
  16.0              
  16.4              
  16.799999999999997
  17.200000000000003
  17.6              
  18.0              
  18.4              
  18.799999999999997
  19.200000000000003
  19.6              
  20.0              

In [84]:
segdict_xe = Dict{String, Any}()
segdict_kr = Dict{String, Any}()
for (c, cage) in enumerate(cages)
    # read in aligned and centered cage
    atoms, x = aligned_and_centered_cage(cage)
    
    # assert snapshot box is big enough
    @assert(all(x .< maximum(x_grid_pts)) && all(x .> minimum(x_grid_pts)),
        "atoms outside snapshot!")
    
    # convert cage to a list of LJSphere's for PorousMaterials.jl
    ljspheres = Atoms(atoms, x)
    
    # preallocate grid; choose origin here solely for visualization
    # when we write to a .cube file. the origin passed to `Grid`
    # is so the cage is centered at the origin in the viz. (checked)
    grid_xe = Grid(snapshot_box, (nb_grid_pts, nb_grid_pts, nb_grid_pts), 
                zeros(Float64, nb_grid_pts, nb_grid_pts, nb_grid_pts), 
                :energy, -center_of_box)
    grid_kr = Grid(snapshot_box, (nb_grid_pts, nb_grid_pts, nb_grid_pts), 
                zeros(Float64, nb_grid_pts, nb_grid_pts, nb_grid_pts), 
                :energy, -center_of_box)
        
    for i = 1:nb_grid_pts, j = 1:nb_grid_pts, k = 1:nb_grid_pts
        # what grid pt is here inside the loop?
        x_grid_pt = [x_grid_pts[i], x_grid_pts[j], x_grid_pts[k]]
        
        # put helium at this grid pt
        translate_to!(xe, x_grid_pt)
        translate_to!(kr, x_grid_pt)
        
        # compute potential energy of He adsorbate here
        energy_xe = vdw_energy_no_PBC(xe, ljspheres, ljff)
        energy_kr = vdw_energy_no_PBC(kr, ljspheres, ljff)
        
        # if framework atom overlaps with cage atom... energy > k T
        grid_xe.data[i, j, k] = energy_xe
        grid_kr.data[i, j, k] = energy_kr
        
    end
    segmented_grid_xe = _segment_grid(grid_xe, 15.0, false)
    segmented_grid_kr = _segment_grid(grid_kr, 15.0, false)
    segdict_xe[cage] = segmented_grid_xe
    segdict_kr[cage] = segmented_grid_kr
end

In [85]:
accessible_xe = trues(length(cages))
accessible_kr = trues(length(cages))
for (c, key) in enumerate(cages)
    global accessible_xe
    global accessible_kr
    segmented_grid_xe = segdict_xe[key]
    segmented_grid_kr = segdict_kr[key]
    N = ceil(Int, nb_grid_pts/2)
    @printf("%6s:\n", key)
    if length(unique(segmented_grid_xe.data)) > 2 && segmented_grid_xe.data[1, 1, 1] != segmented_grid_xe.data[N, N, N]
        @printf("\tXe\n\tOuter Segment: %d\tInner Segment: %d\t Number of segments: %d\n", 
                segmented_grid_xe.data[1,1,1], segmented_grid_xe.data[N,N,N],length(unique(segmented_grid_xe.data)))
        #println(key, "  ", segmented_grid_xe.data[N, N, N], "   ", length(unique(segmented_grid_xe.data)))
        accessible_xe[c] = false
    end
    if length(unique(segmented_grid_kr.data)) > 2 && segmented_grid_kr.data[1, 1, 1] != segmented_grid_kr.data[N, N, N]
        @printf("\tKr\n\tOuter Segment: %d\tInner Segment: %d\t Number of segments: %d\n", 
                segmented_grid_kr.data[1,1,1], segmented_grid_kr.data[N,N,N],length(unique(segmented_grid_kr.data)))
        #println(key, "  ", segmented_grid_kr.data[N, N, N], "   ", length(unique(segmented_grid_kr.data)))
        accessible_kr[c] = false
    end
    if accessible_kr[c] && accessible_xe[c]
        @printf("\t Same segment inside and outside\n\t Number of segments: %d (Xe), %d (Kr)\n", 
                length(unique(segmented_grid_xe.data)), length(unique(segmented_grid_kr.data)))
    end
end

open("inaccessible_cages.txt", "w") do f
    for cage in cages[.!accessible_xe]
        @printf(f, "%s\n", cage)
    end
end
@printf("See inaccessible_cages.txt for cages that are inaccessible towards Xenon\n")

   A11:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
   B11:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
   B13:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
   B15:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
   B18:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
    B1:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
   B23:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
   B24:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
   B25:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
   B26:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
    B2:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
    B4:
	 Same segment inside and outside
	 Number of segments: 2 (Xe), 2 (Kr)
    B5:
	 Same segment inside and outside
	 Number o